In [1]:
def get_box_annotation(class_name, center_x, center_y, width, height):
    box = dict()
    box["displayName"] = class_name
    box["xMin"] = center_x - width / 2
    box["xMax"] = center_x + width / 2
    box["yMin"] = center_y - height / 2
    box["yMax"] = center_y + height / 2
    return box

In [2]:
class_traductor = {0: "biodegradable",
                   1: "clothes",
                   2: "electronic",
                   3: "glass",
                   4: "paper",
                   5: "plastic"}

In [9]:
folders = [("test", "test"),
           ("train", "training"),
           ("valid", "validation")]

In [17]:
import os

json_list = []

for folder, vertex_type in folders:
    root = "Waste_Detection-1"
    files_in_directory = os.listdir(f"{root}/{folder}/images/")
    for file in files_in_directory:
        vertex_dic = dict()
        img = file
        label_name = ".".join(img.split(".")[:-1]) + ".txt"
        label = f"{root}/{folder}/labels/{label_name}"
        
        vertex_dic["imageGcsUri"] = f"gs://cloud-ai-platform-4f807e4a-a411-472d-8205-9769f14f21ef/cv/{img}"
        
        box_annotations = []
        with open(label, 'r') as file_label:
            for line in file_label:
                label_info = line.strip().split(" ")
                box_annotations.append(get_box_annotation(
                    class_traductor[ int(label_info[0]) ], 
                    float(label_info[1]),
                    float(label_info[2]),
                    float(label_info[3]), 
                    float(label_info[4])
            ))
        vertex_dic["boundingBoxAnnotations"] = box_annotations
        vertex_dic["dataItemResourceLabels"] = {"aiplatform.googleapis.com/ml_use": vertex_type}
        
        json_list.append(vertex_dic)
        

In [18]:
len(json_list)

1473

In [19]:
import json
with open("img_labels.jsonl", 'w') as file:
    for entry in json_list:
        json_line = json.dumps(entry)
        file.write(json_line + '\n')